In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from pandas import *
from PIL import Image as im
import matplotlib.image

In [2]:
#Leemos los datos de entrenamiento
DataTrain = read_csv("Smiles_Train.csv")
SmilesTrain = DataTrain['SMILES'].tolist()
LogPStringTrain = DataTrain['logP'].tolist()

#Leemos los datos de prueba
DataTest = read_csv("Smiles_Test.csv")
SmilesTest = DataTest['SMILES_Test'].tolist()
LogPStringTest = DataTest['LogP_Test'].tolist()

#Se convierte en arreglo numerico
LogPTrain = np.array(LogPStringTrain)
LogPTest = np.array(LogPStringTest)

In [3]:
PromTrain, PromTest = 0, 0
SmilesTrainLen, SmilesTestLen = [], []
for x in range(len(SmilesTrain)):
    PromTrain = PromTrain + len(SmilesTrain[x])
    SmilesTrainLen.append(len(SmilesTrain[x]))

for y in range(len(SmilesTest)):
    PromTest = PromTest + len(SmilesTest[y])
    SmilesTestLen.append(len(SmilesTest[y]))

PromTrain = PromTrain / len(SmilesTrain)
PromTest = PromTest / len(SmilesTest)

print(PromTrain, PromTest)
print(min(SmilesTrainLen), max(SmilesTrainLen), min(SmilesTestLen), max(SmilesTestLen))

35.89927172017643 37.885858585858585
1 336 1 1415


In [97]:
MoleculaAsciiTrain, NewLogPTrain = [], []
for x in range(len(SmilesTrain)):
    b = 0
    s = SmilesTrain[x]
    MolLen = len(SmilesTrain[x])
    Arreglo1 = []
    if (MolLen < 64):
        for y in range(64):
            if (b < MolLen):
                Arreglo1.append(ord(s[b]))
            else:
                Arreglo1.append(0)
            b += 1
        MoleculaAsciiTrain.append(Arreglo1)
        NewLogPTrain.append(LogPTrain[x])

In [5]:
MoleculaAsciiTest, NewLogPTest = [], []
for x in range(len(SmilesTest)):
    b = 0
    s = SmilesTest[x]
    MolLen = len(SmilesTest[x])
    Arreglo2 = []
    if (MolLen < 36):
        for y in range(36):
            if (b < MolLen):
                Arreglo2.append(ord(s[b]))
            else:
                Arreglo2.append(0)
            b += 1
        MoleculaAsciiTest.append(Arreglo2)
        NewLogPTest.append(LogPTest[x])

In [98]:
SmilesTrainImgR = []
i=15
for y in range(len(MoleculaAsciiTrain)):
    SmilesTrainImg = []
    SmilesTrainImg = MoleculaAsciiTrain[y]
    SmilesTrainImg = np.array(SmilesTrainImg)
    SmilesTrainImg = np.reshape(SmilesTrainImg, (8,8))
    SmilesTrainImgR.append(SmilesTrainImg)
matplotlib.image.imsave('SmilesTrainImages/Train'+str(i)+'.png',SmilesTrainImgR[i])

In [26]:
SmilesTestImgR = []
for y in range(len(MoleculaAsciiTest)):
    SmilesTestImg = []
    SmilesTestImg = MoleculaAsciiTest[y]
    SmilesTestImg = np.array(SmilesTestImg)
    SmilesTestImg = np.reshape(SmilesTestImg, (6,6))
    SmilesTestImgR.append(SmilesTestImg)
matplotlib.image.imsave('SmilesTestImages/Test'+str(i)+'.png',SmilesTestImgR[i])

In [99]:
SmilesTestImgR = np.asarray(SmilesTestImgR)
SmilesTrainImgR = np.asarray(SmilesTrainImgR)
NewLogPTrain = np.asarray(NewLogPTrain)
NewLogPTest = np.asarray(NewLogPTest)
SmilesTrainImgR = tf.convert_to_tensor(SmilesTrainImgR)
SmilesTestImgR = tf.convert_to_tensor(SmilesTestImgR)
NewLogPTrain = NewLogPTrain.reshape(-1,1)
NewLogPTest = NewLogPTest.reshape(-1,1)

In [100]:
print(SmilesTrainImgR.shape)
print(SmilesTestImgR.shape)
print(NewLogPTrain.shape)
print(NewLogPTest.shape)

(17568, 8, 8)
(1180, 6, 6)
(17568, 1)
(1180, 1)


In [101]:
normalizador = tf.keras.layers.Normalization(axis=-1)
normalizador.adapt(SmilesTrainImgR)

In [102]:
modelo = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=[784]),
    tf.keras.layers.Conv2D(32, (1,1), input_shape=(8,8,1),activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=128, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=128, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=1)
])
modelo.compile(
    optimizer='adam',
    loss='mean_squared_error',
    #loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    #loss = 'binary_crossentropy',
    metrics=['accuracy']
)

In [104]:
print("Iniciando Entrenamiento...")
modelo.fit(
    SmilesTrainImgR, NewLogPTrain, epochs = 10,
    verbose = 1
)
print("Entrenamiento finailzado exitosamente")

Iniciando Entrenamiento...
Epoch 1/10
549/549 [==============================] - 1s 3ms/step - loss: 6.2009 - accuracy: 0.0159
Epoch 2/10
549/549 [==============================] - 2s 3ms/step - loss: 6.0568 - accuracy: 0.0158
Epoch 3/10
549/549 [==============================] - 2s 3ms/step - loss: 6.0056 - accuracy: 0.0159
Epoch 4/10
549/549 [==============================] - 2s 3ms/step - loss: 5.9938 - accuracy: 0.0159
Epoch 5/10
549/549 [==============================] - 2s 3ms/step - loss: 5.9823 - accuracy: 0.0159
Epoch 6/10
549/549 [==============================] - 2s 3ms/step - loss: 5.9943 - accuracy: 0.0159
Epoch 7/10
549/549 [==============================] - 2s 3ms/step - loss: 5.9963 - accuracy: 0.0159
Epoch 8/10
549/549 [==============================] - 1s 3ms/step - loss: 6.0102 - accuracy: 0.0159
Epoch 9/10
549/549 [==============================] - 2s 3ms/step - loss: 6.0150 - accuracy: 0.0159
Epoch 10/10
549/549 [==============================] - 2s 3ms/step - loss